In [1]:
using Pkg, Plots, XLSX, Dates, DataFrames, Random, Statistics
using NBInclude
using DifferentialEquations
using ForwardDiff, Optim 
@nbinclude("ODE_models.ipynb")
@nbinclude("Norm_functions.ipynb")
@nbinclude("HelperFunctions.ipynb")

data_to_df (generic function with 1 method)

In [10]:
function FirstOrderFit(;ODE_model, ODE_vars, data_obs, t_obs, t_all, x0, x_LBs, x_UBs, UBs,
                        param_opt_indices, param_fix_indices, IC_opt_indices, IC_fix_indices,
                        params_fix, ICs_fix, f_calc_ICs, N0, norm, norm_weights, norm_scale,
                        optimizer, make_plots::Bool, integrator_options::Dict,
                        optimizer_options::Dict, constraints::Any = 1)
    
    
    #Note: constraints is just dummy argument in this function. It's only added to easily compatibility
    #based on the way ParamEst decides which optimization method to call. 
    
    #THE ARGUMENTS
    #ODE_model: The function that computes the vector dY/dt given a vector Y, the time t, and parameter values. 
    #ODE_vars: The ODE variable names, given as an array of Strings
    #data: A DataFrame of observed data values, where each column corresponds to an ODE variable.
    #t_obs: An array of the time values at which the data in data_obs were observed.  
    #t_all: An array of time values for which to plot the predicted value 
    #x0: The initial guess (Array of Reals) for the optimizer     
    #x_LBs: The lower bounds for all of the variables being fitted ([param_LBs; IC_LBs])
    #x_UBs: The upper bounds for all the variables being fitted (should be [1,1,1,...,l])
    #UBs: The original (unscaled) upper bounds on all the parameters & ICs being fitted 
    
    ########################################################################
    #Step 1: Get some info needed in order to define the objective function
    ########################################################################
    
    #Scale the observed data 
    data_obs_scaled = data_obs ./ N0
    
    vars_to_fit = names(data_obs)    
    num_params_opt = length(param_opt_indices)
    num_ICs_opt = length(IC_opt_indices)
    
    num_params = num_params_opt + length(param_fix_indices)
    num_ICs_and_IC_ratios = num_ICs_opt + length(IC_fix_indices)
    
    #Unpack integrator options
    rtol = get(integrator_options, :rtol, 1e-8)
    atol = get(integrator_options, :atol, 1e-8)
    integrator = get(integrator_options, :integrator, Tsit5())
    
    #Get the original upper bounds 
    param_UBs = UBs[1:num_params_opt]
    IC_UBs = UBs[num_params_opt+1:end]
    
    #########################################
    #Step 2: Define the objective function
    #########################################
    
    function objective(x)
        
        #Unpack scaled params & ICs/IC ratios
        params_opt_scaled = x[1:num_params_opt]
        IC_ratios_opt_scaled = x[num_params_opt+1:end]
        
        #Scale the params & IC ratios back
        params_opt = params_opt_scaled .* param_UBs
        IC_ratios_opt = IC_ratios_opt_scaled .* IC_UBs
        
        #Create an array to store the ODE parameters
        ODE_params = Array{Real,1}(undef, num_params) 
        
        #Populate ODE_params with the parameter values
        ODE_params[param_opt_indices] = params_opt
        ODE_params[param_fix_indices] = params_fix
        
        #Create an array to store the ICs and IC ratios
        ICs_and_IC_ratios = Array{Real,1}(undef, num_ICs_and_IC_ratios) 
        
        #Populate ICs_and_IC_ratios 
        ICs_and_IC_ratios[IC_opt_indices] = IC_ratios_opt
        ICs_and_IC_ratios[IC_fix_indices] = ICs_fix
        
        #Calculate the ODE Initial Conditions (for the integrator)
        ODE_ICs = f_calc_ICs(ICs_and_IC_ratios)
        
        #Scale the initial condition
        ODE_ICs_scaled = ODE_ICs ./ N0
        
        #Get the time interval for which to solve the ODE
        t_span = 1.0 .* [t_obs[1], t_obs[end]]
        
        #Compute the numerical solution of the ODE  
        ODE_prob = ODEProblem(ODE_model, ODE_ICs_scaled, t_span, ODE_params)
        ODE_sol = solve(ODE_prob, integrator, reltol = rtol, abstol = atol, saveat = t_obs)
        
        #Testing w/fixed time-step:
        #ODE_sol = solve(ODE_prob, integrator, dt = 0.1, reltol = rtol, abstol = atol, saveat = t_obs)
        
        #Convert ODE_sol to a DataFrame (for convenience)
        ODE_sol = DataFrame(ODE_sol', ODE_vars)
        
        #Calculate the residual vectors and then compute their average norm 
        norm_sum = 0.0
        
        for var in vars_to_fit
            var_pred = ODE_sol[:,var]
            var_obs = data_obs_scaled[:,var]
            norm_sum += norm(var_pred, var_obs; w = norm_weights)
        end
        mean_norm = norm_sum / length(vars_to_fit)
        
        return mean_norm 
    end
    #End of Objective Function definition#################################
    
   
    #############################
    #Step 3: Call the Optimizer
    #############################
     
    #Unpack optimizer options 
    xtol_abs = get(optimizer_options, :xtol_abs, 1e-10)
    ftol_rel = get(optimizer_options, :ftol_rel, 1e-4)
    gtol_abs = get(optimizer_options, :gtol_abs, 1e-8)
    max_iter = get(optimizer_options, :max_iter, 1000)
    max_time = get(optimizer_options, :max_time, 3000)
    show_trace = get(optimizer_options, :show_trace, true)
    show_every = get(optimizer_options, :show_every, 5)
  
     ###GradientDescent() w/box constraints only#######################################################    
     od = OnceDifferentiable(objective, x0, autodiff=:forward)
     out = Optim.optimize(od, x_LBs, x_UBs, x0, Fminbox(optimizer),
                          Optim.Options(x_tol = xtol_abs, f_tol = ftol_rel, g_tol = gtol_abs,
                                        iterations = max_iter, time_limit = max_time, 
                                        show_trace = show_trace, show_every = show_every,
                                        allow_f_increases = true))
    #print the output (for testing purposes)
    println(out)
    
    #The optimal parameter values found by the optimizer
    x_opt = out.minimizer 
    
    #Plot the ODE solution computed from x_opt against the observed data 
    if make_plots == true
        #DataFrame of predicted values 
        y = ODE_sol(ODE_model, ODE_vars, x_opt, params_fix, ICs_fix, param_opt_indices, param_fix_indices,
                     IC_opt_indices, IC_fix_indices, param_UBs, IC_UBs, f_calc_ICs, 
                     N0, t_all, int_options)
        
        for var in vars_to_fit
           plot(y.t, y[:,var],
                 size = (450,350),
                 line = :line,
                 lw = 1.5,
                 label = "$var (pred)",
                 legend = :topleft,
                 xlabel = "t",
                 ylabel = "Number",
                 title = "$var")
            
           display(plot!(t_obs, data_obs[:,var],
                 line = :scatter,
                 markersize = 2,
                 markerstrokewidth = 0,
                 label = "$var (obs, fit)"))
        end
    end
    
return (minimizer = out.minimizer, minimum = out.minimum, full_output = out) 
   
    
end


FirstOrderFit (generic function with 1 method)